<a href="https://colab.research.google.com/github/5starkarma/face_smoothing/blob/master/face_smoothing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! git clone https://github.com/5starkarma/face-smoothing.git

In [21]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [22]:
% cd /content/drive/My\ Drive/Colab Notebooks/face-smoothing

/content/drive/My Drive/Colab Notebooks/face-smoothing


In [23]:
pip install -U PyYAML

Requirement already up-to-date: PyYAML in /usr/local/lib/python3.6/dist-packages (5.3.1)


In [118]:
! python3 infer.py --save-steps

Face detected: [113, 152, 464, 612]
Operation ran in 0.2709 seconds


In [83]:
! rm -r /content/drive/My\ Drive/Colab\ Notebooks/face-smoothing/data/output